<a href="https://colab.research.google.com/github/JoWillS12/TugasAkhir/blob/main/DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Import


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#2. Constant and Load DenseNet

In [ ]:
# Constants
IMG_WIDTH, IMG_HEIGHT = 224, 224
NUM_CLASSES = 16  # Number of flower classes
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 10

# Load DenseNet121 pre-trained on ImageNet
base_model = DenseNet121(
    weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
)

#3. Layers And Model


In [ ]:
# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the DenseNet121 model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile model
optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


#4. Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load and preprocess data
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/COMVIS2023/flowers 2',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/COMVIS2023/flowers 2',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/COMVIS2023/flowers 2',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 12701 images belonging to 16 classes.
Found 3166 images belonging to 16 classes.
Found 15867 images belonging to 16 classes.


#5. Train and Testing



In [ ]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=NUM_EPOCHS
)

# Save the trained model
model.save('flower_classification_model.h5')

# Load the saved model
loaded_model = load_model('flower_classification_model.h5')

# Print the loaded model summary
loaded_model.summary()

Epoch 1/10
396/396 [==============================] - 6538s 16s/step - loss: 0.5322 - accuracy: 0.8328 - val_loss: 0.3542 - val_accuracy: 0.8871
Epoch 2/10
396/396 [==============================] - 77s 194ms/step - loss: 0.2670 - accuracy: 0.9144 - val_loss: 0.3175 - val_accuracy: 0.8986
Epoch 3/10
396/396 [==============================] - 68s 171ms/step - loss: 0.1988 - accuracy: 0.9376 - val_loss: 0.2880 - val_accuracy: 0.9101
Epoch 4/10
396/396 [==============================] - 76s 192ms/step - loss: 0.1567 - accuracy: 0.9481 - val_loss: 0.3054 - val_accuracy: 0.9098
Epoch 5/10
396/396 [==============================] - 75s 190ms/step - loss: 0.1208 - accuracy: 0.9605 - val_loss: 0.4336 - val_accuracy: 0.8855
Epoch 6/10
396/396 [==============================] - 69s 175ms/step - loss: 0.1008 - accuracy: 0.9653 - val_loss: 0.3491 - val_accuracy: 0.9037
Epoch 7/10
396/396 [==============================] - 77s 194ms/step - loss: 0.0768 - accuracy: 0.9747 - val_loss: 0.3016 - val_ac

#6. Result



In [12]:
# Evaluate the model on the test set
test_loss, test_accuracy = loaded_model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

495/495 [==============================] - 73s 147ms/step - loss: 0.1032 - accuracy: 0.9737
Test Loss: 0.10318212956190109
Test Accuracy: 0.973737359046936
